In [4]:
%pip install radcad
%pip install pandas
%pip install numpy
%pip install plotly

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
from radcad import Model, Simulation, Experiment
import math
import pandas as pd
import plotly
from numpy import random
from functools import partial

from models import Stake, LiquidityPool, SingleStaker

def _update_from_signal(
    state_variable,
    signal_key,
    params,
    substep,
    state_history,
    previous_state,
    policy_input,
):
    return state_variable, policy_input[signal_key]


def update_from_signal(state_variable, signal_key=None):
    """A generic State Update Function to update a State Variable directly from a Policy Signal
    Args:
        state_variable (str): State Variable key
        signal_key (str, optional): Policy Signal key. Defaults to None.
    Returns:
        Callable: A generic State Update Function
    """
    if not signal_key:
        signal_key = state_variable
    return partial(_update_from_signal, state_variable, signal_key)


random.seed(1234)

initial_state = {
    # this is heimdall's vault
    'rune_vault': 0.0,
    'asset_vault': 0.0,
    # this is the thorchain pool
    'rune_pool': LiquidityPool(token='rune', value=400.0, shares=400.0),
    'asset_pool': LiquidityPool(token='asset', value=1.0, shares=1.0),
    # these refer to heimdall's stake in the rune pool. We also need to keep track of shares in the pool
    'rune_stake': 0.0,
    'asset_stake': 0.0,
    # the `staked` below is a single sided stake amount, and these are single sided stakers.
    # lp_ratio is a placeholder to indicate share of the lp'ed amount
    'asset_stakers': [SingleStaker(token='asset', wallet=10.0, lp_ratio=0.0, staked=0.0) for i in range(10)],
    'rune_stakers': [SingleStaker(token='rune', wallet=10000.0, lp_ratio=0.0, staked=0.0) for i in range(10)],
}

params = {
    # price in USD to calculate value of gas fee.
    'asset_price': [2000.0],
    'rune_price': [5.0],
    'thor_chain_tx_fees': [0.2],
    'asset_chain_tx_fees': [0.01],
    'lp_threshold': [1000.0],
    'max_swap_amount': [50.0],
    'asset_stake_amount': [1500],
    'rune_stake_amount': [800]
}

initial_state['asset_pool'].value

1.0

In [6]:
def to_rune(val_usd, params):
    return val_usd / params['rune_price']

def to_asset(val_usd, params):
    return val_usd / params['asset_price']

def from_rune(val_rune, params):
    # returns USD value
    return val_rune * params['rune_price']

def from_asset(val_asset, params):
    # returns USD value
    return val_asset * params['asset_price']

def swap_to_pool(swap_amount_from, from_pool, to_pool):
    return ((swap_amount_from * from_pool * to_pool) / ((swap_amount_from + from_pool) ** 2))

def swap_to_rune(swap_amount_asset, previous_state):
    return swap_to_pool(swap_amount_asset, previous_state['asset_pool'].value, previous_state['rune_pool'].value)

def swap_to_asset(swap_amount_rune, previous_state):
    return swap_to_pool(swap_amount_rune, previous_state['rune_pool'].value, previous_state['asset_pool'].value)

def lp_fees(swap_amount_from, from_pool, to_pool, to_tx_fees):
    return to_tx_fees + ((swap_amount_from**2)*to_pool/(swap_amount_from+from_pool)**2)
    
def lp_fees_to_asset(swap_amount_rune, params, previous_state):
    return lp_fees(swap_amount_rune, previous_state['rune_pool'].value, previous_state['asset_pool'].value, params['asset_chain_tx_fees'])    
    
def lp_fees_to_rune(swap_amount_asset, params, previous_state):
    return lp_fees(swap_amount_asset, previous_state['asset_pool'].value, 
                   previous_state['rune_pool'].value, params['thor_chain_tx_fees'])

In [7]:
def do_arb(rune_pool, asset_pool, rune_price, asset_price, rune_fees, asset_fees):
    precision = 0.0001
    # adapted from https://gitlab.com/thorchain/trade-bots/samaritan/-/blob/master/src/samaritan.py
    ratio = rune_price / asset_price
    # this is the amount of rune / asset that would need to be added to match the pools price with the external price, 
    # keeping the other pool constant
    expected_rune = (asset_pool / ratio) - rune_pool
    expected_asset = (rune_pool * ratio) - asset_pool
    
    _from, expected = ("rune", expected_rune) if expected_rune > 0 else ("asset", expected_asset)    
    from_pool, from_price, from_fees, to_pool, to_price, to_fees = (rune_pool, rune_price, rune_fees, asset_pool, asset_price, asset_fees) \
    if _from == "rune" else (asset_pool, asset_price, asset_fees, rune_pool, rune_price, rune_fees)

    # The original samaritan bot finds a swap with a min profit with the largest trade possible
    # 
    # Here, find the swap with max profit, to a certain precision.
    # To maximise profit, would have to go lower to find max profit to the required precision.
    todo = {
        "profit": None,
        "to_sell": None,
        "emissions": None,
    }
    
    for i in range(0, 10000):
        # we start at the expected / 2. When you swap in an asset, the other side will go down.
        # if the exchange rate stays constant as the amount swapped increases, the expected / 2 would be the optimal amount.
        # but thorchain increase slippage as the amount goes up, so the amount will be lesser than that.
        # We just brute force to find max amount that generates a profit. 
        to_sell = abs(expected) * (1 - (i * precision))
        to_pool = asset_pool if _from == "rune" else rune_pool
        
        emission = swap_to_pool(to_sell, from_pool, to_pool)
        
        # the good samaritan bot code doesnt seem to take tx fees into account, it's doing BNB / RUNE so perhaps it doesnt matter?        
        profit = (emission * to_price) - (to_sell * from_price) - rune_fees - asset_fees

        if todo['profit'] is None or profit > todo['profit']:
            todo['profit'] = profit
            todo['to_sell'] = to_sell
            todo['emissions'] = emission
#             print("{}: {}".format(todo, _from))
        else:
            # we break if the new profit isnt greated than the previous profit.
            # given the shape of the curve eg: https://www.wolframalpha.com/input?i=x*%2825*20%2F%28x%2B20%29%5E2%29+-+x*0.8+%29
            # there is only 1 maxima.
            # here we limit the resolution to changes of `precision` 
            break
    
    from_change = todo['to_sell']
    to_change = todo['emissions'] + 2 * to_fees
    new_from_pool = from_pool+ from_change
    new_to_pool = to_pool-to_change
    
    _lp_fees = lp_fees(todo['to_sell'], from_pool, to_pool, to_fees) * to_price
                        
    return {
        'from': _from, # rune | asset 
        'amount': todo['to_sell'], # amount to swap
        'profit': todo['profit'],
        'emissions': todo['emissions'],
        # values after swap

        'rune_fees': rune_fees,
        'asset_chain_fees': asset_fees,
        'treasury_fees': to_fees,
        'lp_fees': _lp_fees,
        'asset_pool_change': -to_change if _from == "rune" else from_change,
        'rune_pool_change': from_change if _from == "rune" else -to_change,
        
        'rune_pool': new_from_pool if _from == "rune" else new_to_pool,
        'asset_pool': new_to_pool if _from == "rune" else new_from_pool,
    }

def arb_bots(rune_pool, asset_pool, rune_price, asset_price, rune_fees, asset_fees):
    arb = do_arb(rune_pool, asset_pool, rune_price, asset_price, rune_fees, asset_fees)
    arbs = []
    while(arb['profit'] is not None and arb['profit'] > 0):
        arbs.append(arb)
        new_rune_pool = arb['rune_pool']
        new_asset_pool = arb['asset_pool']        
        arb = do_arb(new_rune_pool, new_asset_pool, rune_price, asset_price, rune_fees, asset_fees)
    return arbs



arb_bots(20.0,25.0,1.0,0.9, 0.001,0.001)

[{'from': 'rune',
  'amount': 0.5932499999999998,
  'profit': 0.03425687430445168,
  'emissions': 0.6994520825605016,
  'rune_fees': 0.001,
  'asset_chain_fees': 0.001,
  'treasury_fees': 0.001,
  'lp_fees': 0.01957274765905579,
  'asset_pool_change': -0.7014520825605016,
  'rune_pool_change': 0.5932499999999998,
  'rune_pool': 20.59325,
  'asset_pool': 24.2985479174395},
 {'from': 'rune',
  'amount': 0.3105704011068661,
  'profit': 0.0075080629064049895,
  'emissions': 0.3556427377925234,
  'rune_fees': 0.001,
  'asset_chain_fees': 0.001,
  'treasury_fees': 0.001,
  'lp_fees': 0.0057271592368504836,
  'asset_pool_change': -0.3576427377925234,
  'rune_pool_change': 0.3105704011068661,
  'rune_pool': 20.903820401106866,
  'asset_pool': 23.940905179646975},
 {'from': 'rune',
  'amount': 0.15862668408395358,
  'profit': 0.00042576485697329476,
  'emissions': 0.17894716548991874,
  'rune_fees': 0.001,
  'asset_chain_fees': 0.001,
  'treasury_fees': 0.001,
  'lp_fees': 0.0021221314309487053

In [20]:
def p_add_lp(params, substep, state_history, previous_state):
    asset_threshold = to_asset(params['lp_threshold'], params)
    rune_threshold = to_rune(params['lp_threshold'], params)
    if (previous_state['asset_vault'] > asset_threshold + params['asset_chain_tx_fees']) and \
        (previous_state['rune_vault'] > rune_threshold + params['thor_chain_tx_fees']):
        return {
            'rune_fees': 0,
            'asset_chain_fees': params['asset_chain_tx_fees'],
            'thor_chain_fees': params['thor_chain_tx_fees'],
            'treasury_fees': 0,
            'lp_fees': 0,
            'asset_vault_change': -(asset_threshold + params['asset_chain_tx_fees']),
            'rune_vault_change': -(rune_threshold + params['thor_chain_tx_fees']),
            'asset_pool_change': asset_threshold,
            'rune_pool_change': rune_threshold,
            'asset_pool_shares_change': asset_threshold,
            'rune_pool_shares_change': rune_threshold,
            'asset_stake_change': asset_threshold,
            'rune_stake_change': rune_threshold
        }
    else:
        return {}

def p_swap_to_rune(params, substep, state_history, previous_state):
    swap_amount_asset = to_asset(random.rand() * params['max_swap_amount'], params)
    amount_rune = swap_to_rune(swap_amount_asset, previous_state)
    if swap_amount_asset > previous_state['asset_pool'].value:
        return {
            'rune_fees': 0,
            'asset_chain_fees': 0,
            'treasury_fees': 0,
            'lp_fees': 0,
            'asset_pool_change': 0,
            'rune_pool_change': 0,
        }
    
    return {
        'rune_fees': params['thor_chain_tx_fees'],
        'asset_chain_fees': params['asset_chain_tx_fees'],
        'treasury_fees': params['thor_chain_tx_fees'],
        'lp_fees': from_rune(lp_fees_to_rune(swap_amount_asset, params, previous_state), params),
        'asset_pool_change': swap_amount_asset,
        'rune_pool_change': -(amount_rune + 2 * params['thor_chain_tx_fees']) # add treasury and chain fees
    }

def p_swap_to_asset(params, substep, state_history, previous_state):
    swap_amount_rune = to_rune(random.rand() * params['max_swap_amount'], params)
    amount_asset = swap_to_asset(swap_amount_rune, previous_state)
    if swap_amount_rune > previous_state['rune_pool'].value:
        return {
            'rune_fees': 0,
            'asset_chain_fees': 0,
            'treasury_fees': 0,
            'lp_fees': 0,
            'asset_pool_change': 0,
            'rune_pool_change': 0,
        }
    
    return {
        'rune_fees': params['thor_chain_tx_fees'],
        'asset_chain_fees': params['asset_chain_tx_fees'],
        'treasury_fees': params['asset_chain_tx_fees'],
        'lp_fees': from_asset(lp_fees_to_asset(swap_amount_rune, params, previous_state), params),
        'asset_pool_change': -(amount_asset + 2 * params['asset_chain_tx_fees']), # add treasury and chain fees,
        'rune_pool_change': swap_amount_rune
    }

def p_remove_lp(params, substep, state_history, previous_state):
    
    
    return {
        'rune_fees': 0,
        'asset_chain_fees': 0,
        'treasury_fees': 0,
        'lp_fees': 0,
        'asset_pool_change': 0,
        'rune_pool_change': 0,
        'asset_vault_change': 0,
        'rune_vault_change': 0,
        'asset_stake_change': 0,
        'rune_stake_change': 0,
    }

def p_ilp_bonus(params, substep, state_history, previous_state):
    #TODO. Only applicable when removing.
    return {
        'rune_pool_change': 0,
    }

def p_add_to_vault(params, substep, state_history, previous_state):
    new_asset_deposit = 0
    new_rune_deposit = 0
    asset_staker_changes = [0] * len(previous_state['asset_stakers'])
    rune_staker_changes = [0] * len(previous_state['rune_stakers'])
        
    if (previous_state['timestep'] % 10 == 0):
        new_asset_deposit = to_asset(params['asset_stake_amount'], params)
        asset_staker = int(previous_state['timestep'] % 10)
        asset_staker_changes[asset_staker] = -new_asset_deposit

    if (previous_state['timestep'] % 5 == 0):
        new_rune_deposit = to_rune(params['rune_stake_amount'], params)
        rune_staker = int(previous_state['timestep'] % 10)
        rune_staker_changes[rune_staker] = -new_rune_deposit
    
    
    return {
        'asset_vault_change': new_asset_deposit,
        'rune_vault_change': new_rune_deposit,
        'asset_staker_changes': asset_staker_changes,
        'rune_staker_changes': rune_staker_changes
    }

def p_arb_bots(params, substep, state_history, previous_state):
    target_price = params['asset_price'] / params['rune_price']
    current_price = previous_state['rune_pool'].value / previous_state['asset_pool'].value
    transactions = arb_bots(
        previous_state['rune_pool'].value, previous_state['asset_pool'].value,
        params['rune_price'], params['asset_price'], params['thor_chain_tx_fees'],
        params['asset_chain_tx_fees']
    )
    return {
        'rune_fees': sum(t['rune_fees'] for t in transactions),
        'asset_chain_fees': sum(t['asset_chain_fees'] for t in transactions),
        'treasury_fees': sum(t['treasury_fees'] for t in transactions),
        'lp_fees': sum(t['lp_fees'] for t in transactions),
        'asset_pool_change': sum(t['asset_pool_change'] for t in transactions),
        'rune_pool_change': sum(t['rune_pool_change'] for t in transactions)
    }

In [21]:
def s_asset_vault(params, substep, state_history, previous_state, policy_input):
    return 'asset_vault', previous_state['asset_vault'] + policy_input['asset_vault_change']

def s_rune_vault(params, substep, state_history, previous_state, policy_input):    
    return 'rune_vault', previous_state['rune_vault'] + policy_input['rune_vault_change']

def s_asset_pool(params, substep, state_history, previous_state, policy_input):
    return 'asset_pool', LiquidityPool(
        token=previous_state['asset_pool'].token,
        value=previous_state['asset_pool'].value + policy_input['asset_pool_change'],
        shares=previous_state['asset_pool'].shares + policy_input.get('asset_pool_shares_change', 0)
    )

def s_rune_pool(params, substep, state_history, previous_state, policy_input):
    return 'rune_pool', LiquidityPool(
        token=previous_state['rune_pool'].token,
        value=previous_state['rune_pool'].value + policy_input['rune_pool_change'],
        shares=previous_state['rune_pool'].shares + policy_input.get('rune_pool_shares_change', 0)
    )
        

def s_rune_stake(params, substep, state_history, previous_state, policy_input):
    return 'rune_stake', previous_state['rune_stake'] + policy_input['rune_stake_change']

def s_asset_stake(params, substep, state_history, previous_state, policy_input):
    return 'asset_stake', previous_state['asset_stake'] + policy_input['asset_stake_change']

def update_stakers(stakers, staker_changes):
    return [SingleStaker(
        token=stakers[i].token,
        wallet=stakers[i].wallet + staker_changes[i],
        staked=-staker_changes[i],
        lp_ratio=0) for i in range(len(stakers))]
    
def s_asset_stakers(params, substep, state_history, previous_state, policy_input):
    return 'asset_stakers', update_stakers(previous_state['asset_stakers'], policy_input['asset_staker_changes'])

def s_rune_stakers(params, substep, state_history, previous_state, policy_input):
    return 'rune_stakers', update_stakers(previous_state['rune_stakers'], policy_input['rune_staker_changes'])


In [22]:
state_update_blocks = [
    {
        'policies': {
            'p_arb_bots': p_arb_bots,
        },
        'variables': {
            'asset_pool': s_asset_pool,
            'rune_pool': s_rune_pool,            
        }
    },
    {
        'policies': {
            'add_lp': p_add_lp,
            'remove_lp': p_remove_lp,
            'add_to_vault': p_add_to_vault,
            # does this belong here given that it flows from remove_lp
#             'ilp_bonus': p_ilp_bonus,
        },
        'variables': {
            'asset_vault': s_asset_vault,
            'rune_vault': s_rune_vault,
            'asset_pool': s_asset_pool,            
            'rune_pool': s_rune_pool,
            'asset_stake': s_asset_stake,
            'rune_stake': s_rune_stake,
            'asset_stakers': s_asset_stakers,
            'rune_stakers': s_rune_stakers
        }
    },
    {
        'policies': {
            'swap_to_rune': p_swap_to_rune,
            'swap_to_asset': p_swap_to_asset,
        },
        'variables': {
            'asset_pool': s_asset_pool,
            'rune_pool': s_rune_pool,            
        }
    },
]

In [23]:
model = Model(
    # Model initial state
    initial_state=initial_state,
    # Model Partial State Update Blocks
    state_update_blocks=state_update_blocks,
    # System Parameters
    params=params
)

simulation = Simulation(
    model=model,
    timesteps=200,  # Number of timesteps
    runs=1  # Number of Monte Carlo Runs
)

# Executes the simulation, and returns the raw results
result = simulation.run()

df = pd.DataFrame(result)

In [33]:
asset_price = params['asset_price'][-1] 
rune_price = params['rune_price'][-1]
pools_df = (df.assign(rune_pool_value = lambda df: df.rune_pool.map(lambda pool: (pool.value * rune_price)))
.assign(asset_pool_value = lambda df: df.asset_pool.map(lambda pool: (pool.value * asset_price)))
.assign(asset_stake_value = lambda df: df.asset_stake * asset_price)
.assign(rune_stake_value = lambda df: df.rune_stake * rune_price)
.assign(asset_pool_profit = lambda df: df.asset_pool_value - df.asset_stake_value)
.assign(rune_pool_profit = lambda df: df.rune_pool_value - df.rune_stake_value)
)

In [35]:
import plotly.express as px

fig = px.line(
    pools_df,
    x='timestep',
    y=['rune_pool_value', 'asset_pool_value', 'rune_pool_profit', 'asset_pool_profit', 'rune_stake_value', 'asset_stake_value'],
    facet_row='simulation',
    facet_col='run',
    height=800,
    template='seaborn'
)

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)

fig.show()